In [1]:
import os
import numpy
import gzip
from datetime import datetime, timedelta

In [2]:
WORKSPACE_FOLDER = os.getcwd()
AHI_LST_FOLDER = '/disk2/Data/AHILST_YY'

In [3]:
START_TIME = '2018-01-01T00:00:00Z' # local time
END_TIME = '2019-12-31T23:59:59Z'

UTC_OFFSET = 9 # hour
time_internal = 10*6  # mins

pixel_size = 0.02 # 0.02°
# AMATERASS extent
lst_extent = (120.0, 150.0, 20, 50) # l_lon, r_lon, b_lat, t_lat

In [4]:
tky_site_loc = (36.146, 137.423) # lat, lon
crk_site_loc = (38.201, 127.251) # lat, lon
gck_site_loc = (37.748, 127.162) # lat, lon
gdk_site_loc = (37.749, 127.149) # lat, lon
site_names = ['TKY','CRK', 'GCK', 'GDK']
site_locs = [tky_site_loc, crk_site_loc, gck_site_loc, gdk_site_loc]

In [7]:
def read_AHILST_data(ahi_lst_gz, site_locs):
    if os.path.exists(ahi_lst_gz):
        try:
            with gzip.open(ahi_lst_gz, 'rb') as file:
                gz_data = file.read()
                lst_data = numpy.copy(numpy.frombuffer(gz_data, dtype='f4').reshape(1500, 1500)) # 0.02°

                site_array = []
                for site_loc in site_locs:
                    site_lat_idx = int((lst_extent[3] - site_loc[0])/pixel_size)
                    site_lon_idx = int((site_loc[1] - lst_extent[0])/pixel_size)
                    site_v = lst_data[site_lat_idx][site_lon_idx]
                    if site_v == -999:
                        site_v = numpy.NaN
                    site_array.append(site_v)
                return numpy.array(site_array)
        except Exception as e:
            print(ahi_lst_gz)
            print(e)
            nan_array = numpy.zeros(4,)
            nan_array[nan_array==0] = numpy.NaN
            return nan_array
    else:
        nan_array = numpy.zeros(4,)
        nan_array[nan_array==0] = numpy.NaN
        return nan_array

In [8]:
utc_start_date = datetime.strptime(START_TIME, "%Y-%m-%dT%H:%M:%SZ") - timedelta(hours=UTC_OFFSET)
utc_end_date = datetime.strptime(END_TIME, "%Y-%m-%dT%H:%M:%SZ") - timedelta(hours=UTC_OFFSET)

site_record = []
for site_idx in range(len(site_names)):
    site_record.append([])
temp_date = utc_start_date
while temp_date < utc_end_date:
    current_time_str = temp_date.strftime("%Y%m%d%H%M")
    month_folder = temp_date.strftime("%Y%m")
    day_folder = temp_date.strftime("%Y%m%d")
    file_path = AHI_LST_FOLDER + '/' + month_folder + '/AHILST.v0.' + current_time_str + '.dat.gz'
    site_vs = read_AHILST_data(file_path, site_locs)
    
    this_day_date = temp_date + timedelta(hours=UTC_OFFSET)
    this_day_time = this_day_date.strftime("%Y-%m-%dT%H:%M:%SZ")
    for site_idx in range(len(site_names)):
        site_record[site_idx].append([this_day_time, str(site_vs[site_idx])])
    
    temp_date = temp_date + timedelta(minutes=time_internal)

In [9]:
for site_idx in range(len(site_names)):
    site_name = site_names[site_idx]
    site_record_item = numpy.array(site_record[site_idx])
    numpy.savetxt(os.path.join(WORKSPACE_FOLDER, site_name+'_AHI_LST_hourly'+'.csv'), site_record_item, delimiter=",", fmt='%s')